orders and gvm kitchens

In [ ]:
# ## for jupyter notebook only
# import warnings
# warnings.simplefilter('ignore') ## Importing libraries

# from IPython.display import display, HTML
# display(HTML(data="""
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 95%; }
#     div#maintoolbar-container { width: 95%; }
# </style>
# """))


# Importing libraries
import pandas as pd
import numpy as np
import os
import pygsheets
from ds.utilities.io import ds_trino as ds_presto
from os import listdir
from os.path import isfile, join
from google.cloud import storage
from datetime import timedelta, date, datetime 
import json
import requests
import urllib

# pd.options.display.max_columns = None
# pd.options.display.max_rows = 50
# pd.options.mode.chained_assignment = None

conn = ds_presto.create_trino_connection()

#g_sheets_creds_path = '/home/ruben.lopez/creds_gui.json'


In [ ]:
def facility(x):
    try:
        if x.strip() == 'BR - Sao Paulo - Clélia':
            return 'SAO - Lapa Clelia'
        elif x.strip() == 'BR - Sao Paulo - Acre':
            return 'SAO - Rua do Acre'
        elif x.strip() == 'SA-BR-RJ-José Bonifacio':
            return 'RIO - Jose Bonifacio'
        elif x.strip() == 'BR - Sao Paulo - Guararapes':
            return 'SAO - Guararapes (Brooklin)'
        elif x.strip() == 'BR - Sao Paulo - Dino':
            return 'SAO - Dino'
        elif x.strip() == 'SA-BR-BH-Av General David Sarnoff':
            return 'BHZ - Contagem Industrial'
        elif x.strip() == 'SA-BR-BH-Av-Artur-Guimarães':
            return 'BHZ - Arthur Guimaraes'
        elif x.strip() == 'SA-BR-BH-Rua Paracatu':
            return 'BHZ - Paracatu'
        elif x.strip() == 'BR - Sao Paulo - Borba':
            return 'SAO - Borba'
        elif x.strip() == 'SA-BR-RJ-Uptown':
            return 'RIO - Barra'
        elif x.strip() == 'SA-BR-RJ-Camerino':
            return 'RIO - Camerino'
        elif x.strip() == 'Tancredo Neves':
            return 'BHZ - Tancredo Neves'
        elif x.strip() == 'SA-CO-Bogotá-Castellana':
            return 'BOG - Castellana'
        elif x.strip() == 'SA-CO-Cali-Cali Norte':
            return 'CLO - Chipichape'
        elif x.strip() == 'SA-CO-BAQ-Alto Prado':
            return 'BAQ - Alto Prado'
        elif x.strip() == 'SA-CO-Bogotá-Atabanza':
            return 'BOG - Atabanza'
        elif x.strip() == 'SA-CO-Bogota-San Felipe':
            return 'BOG - San Felipe'
        elif x.strip() == 'SA-CO-Bogotá-Puente Aranda':
            return 'BOG - Puente Aranda'
        elif x.strip() == 'SA-BR-RJ-Niteroi':
            return 'RIO - Niteroi'
        elif x.strip() == 'SA-BR-RJ-Tijuca':
            return 'RIO - Tijuca' 
        elif x.strip() == 'SA-BR-SP-Guarulhos':
            return 'SAO - Espanhola'      
        elif x.strip()=='EM Salitre 2':
            return 'BOG - Salitre 2 EM'
        elif x.strip()=='DeModa Poblado':
            return 'MED - DModa'
        else:
            
            return np.nan
    except Exception as e:
        print(e)
        return np.nan
      

    
def city(x):
    try:
        belo = ['BHZ - Contagem Industrial', 'BHZ - Paracatu', 'BHZ - Arthur Guimaraes', 'BHZ - Tancredo Neves']
        sao = ['SAO - Rua do Acre', 'SAO - Dino', 'SAO - Guararapes (Brooklin)', 'SAO - Lapa Clelia', 'SAO - Borba', 'SAO - Vargas','SAO - Espanhola']
        rio = ['RIO - Jose Bonifacio', 'RIO - Barra', 'RIO - Camerino', 'RIO - Niteroi','RIO - Tijuca' ]
        bog = ['BOG - Prado', 'BOG - Castellana', 'BOG - Cocinas de la 73', 'BOG - Puente Aranda', 'BOG - Atabanza', 'BOG - San Felipe','BOG - Salitre 2 EM']
        cal = ['CLO - Chipichape']
        bar = ['BAQ - Alto Prado']
        med = ['MED - DModa']
        if x.strip() in belo:
            return 'Belo Horizonte'
        elif x.strip() in sao:
            return 'Sao Paulo'
        elif x.strip() in rio:
            return 'Rio de Janeiro'
        elif x.strip() in bog:
            return 'Bogota'
        elif x.strip() in cal:
            return 'Cali'
        elif x.strip() in med:
            return 'Medellin'
        elif x.strip() in bar:
            return 'Barranquilla'
        else:
            return np.nan
    except:
        return np.nan
    
    

In [3]:
query_sin_filtro="""
WITH first_order as (
SELECT 
store_id, 
MIN(reference_time/1000) as first_order

FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
WHERE day_partition >= '2021-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
        
            
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
            '4364cfad-dd86-4232-8164-cc9d51af9ec6', --sjo heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Espanhola
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
           'a63559d1-93c6-4f60-b683-1e207ccda10a' , -- SAO - Batista
           '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
             
            
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1
            )
            
, last_order as (
SELECT 
store_id, 
MAX(reference_time/1000) as last_order
FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
WHERE day_partition >= '2021-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
       
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
            '4364cfad-dd86-4232-8164-cc9d51af9ec6', --sjo heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
             'a63559d1-93c6-4f60-b683-1e207ccda10a' , -- SAO - Batista

        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1
            )
            
, aov_fix as (

SELECT 
dc.country as country,
store_id,
AVG(subtotal) aov_fix

FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
LEFT JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
    
WHERE day_partition >= '2021-01-01'
AND (subtotal is not null or subtotal != 0)
AND NOT is_cancelled
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
       
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
        -- MEDELLIN
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
            '4364cfad-dd86-4232-8164-cc9d51af9ec6', --sjo heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            'a63559d1-93c6-4f60-b683-1e207ccda10a' , -- SAO - Batista
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1, 2

)

SELECT 
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
--date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_week,
'LATAM' as region,
dc.country,
(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849',   -- LIM - Magdalena
         '4e36823e-98a2-4382-8797-48ef313196a9' -- LIM - Lince
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --BOG salitre
      )
   THEN
      'Bogota'
   WHEN 
      facility_id in 
      (
         '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' -- MED - DModa
      )
   THEN
       'Medellin'
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9',  -- SJO - Pavas
         '035c0131-50e4-465a-9596-ad8eb9be03f8', --sjo heredia
         '4364cfad-dd86-4232-8164-cc9d51af9ec6'-- SJO - Heredia
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee'   -- GDL - Av. Mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         'a63559d1-93c6-4f60-b683-1e207ccda10a'  -- SAO - Batista
      )
   then
      'Sao Paulo'
    WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
   
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
  WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   when facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id= '89cf86b9-9e74-4657-86df-2a851492af88' Then  'SAO - Guarulhos'
   WHEN facility_id= 'a63559d1-93c6-4f60-b683-1e207ccda10a'  THEN 'SAO - Batista'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
   
   
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   WHEN facility_id = '4e36823e-98a2-4382-8797-48ef313196a9' THEN 'LIM - Lince'
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   WHEN facility_id = '035c0131-50e4-465a-9596-ad8eb9be03f8' THEN 'SJO - Heredia'
   WHEN facility_id = '4364cfad-dd86-4232-8164-cc9d51af9ec6' THEN 'SJO - Heredia'
  
  WHEN facility_id = '757182ff-7a54-4571-91d4-0027253c0117' THEN 'BOG - Salitre 2 EM'
   
   

   
   
   
END
) AS facility_name,
organization_name,
organization_id,
tenant_type,
fo.first_order,
lo.last_order,
-- date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) < date_add('week', 4, date_trunc('week', DATE(from_unixtime(first_order, facility_timezone)))) as is_new_store,

COUNT (DISTINCT order_id) AS all_orders,
COUNT (DISTINCT CASE WHEN is_cancelled THEN order_id END) AS cancelled_orders,
COUNT (DISTINCT order_id)-COUNT (DISTINCT CASE WHEN is_cancelled THEN order_id END) AS orders,
SUM(coalesce(subtotal, ao.aov_fix)) AS all_gmv,
SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv,
SUM(coalesce(subtotal, 0)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS gmv,
(SUM(coalesce(subtotal, ao.aov_fix)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, ao.aov_fix) ELSE 0 END)) / (COUNT (DISTINCT order_id)-COUNT (DISTINCT case when is_cancelled  THEN order_id END)) AS aov,
SUM(usd_subtotal ) AS all_gmv_usd,
SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END) AS cancelled_gmv_usd,
SUM(usd_subtotal )-SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END) AS gmv_usd,
(SUM(usd_subtotal )-SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END)) / (COUNT (DISTINCT order_id)-COUNT (DISTINCT CASE WHEN is_cancelled = TRUE THEN order_id END)) AS aov_usd,
SUM(CASE WHEN NOT is_cancelled THEN coalesce(net_payout, 0) ELSE 0 END) AS net_payout,
SUM(CASE WHEN NOT is_cancelled THEN abs(discount) ELSE 0 END) AS discount,
(SUM(coalesce(subtotal, 0)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END)) - SUM(CASE WHEN NOT is_cancelled THEN abs(discount) ELSE 0 END) AS net_gmv




FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
LEFT JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
LEFT JOIN first_order fo 
   ON co.store_id=fo.store_id
LEFT JOIN last_order lo 
   ON co.store_id=lo.store_id
LEFT JOIN aov_fix ao 
   ON co.store_id=ao.store_id AND dc.country = ao.country
    
WHERE day_partition >= '2021-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'

AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117',  --'BOG - Salitre 2 EM'
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
        -- MEDELLIN
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- HEREDIA
            '4364cfad-dd86-4232-8164-cc9d51af9ec6',--heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            'a63559d1-93c6-4f60-b683-1e207ccda10a' , -- SAO - Batista
            
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1,2,3,4,5,6,7,8,9,10
ORDER BY 1 DESC,2 ,4,5,6,7




"""


print('Querying Latam Orders & GMV...')
#latam = ds_presto.fetch_data(conn=conn, sql_str=query, use_cache=False)
print('Latam Orders & GMV Pulled ...')



Querying Latam Orders & GMV...
Latam Orders & GMV Pulled ...


In [4]:
query="""

WITH first_order as (
SELECT 
store_id, 
MIN(reference_time/1000) as first_order

FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
WHERE day_partition >= '2021-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
            'bc2ab3e9-e10d-420a-a3d0-395576fed377', --lim la molina
            '993e277c-6d80-428c-8c8f-d2c9857eb8c1',  --LIM - San Borja
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
        
            
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
            '4364cfad-dd86-4232-8164-cc9d51af9ec6', --sjo heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta
            '586ab8c0-f7fa-4bb0-8eac-fde038f41ee2',-- SCL - Nunoa
            '55b113a6-baf1-4ab3-9e26-3117180de4af',--SCL - Lo Barnechea

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
            
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Espanhola
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
           'a63559d1-93c6-4f60-b683-1e207ccda10a' , -- SAO - Batista
           '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
           '812e6913-d78e-4dbc-877d-16993853aa7e', -- sao domingos
           '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
           '89cf86b9-9e74-4657-86df-2a851492af88', --sao espahola
             
            
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1
            )
            
, last_order as (
SELECT 
store_id, 
MAX(reference_time/1000) as last_order
FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
WHERE day_partition >= '2021-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
            ' bc2ab3e9-e10d-420a-a3d0-395576fed377', --lim la molina
             '993e277c-6d80-428c-8c8f-d2c9857eb8c1',  --LIM - San Borja
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
       
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
            '4364cfad-dd86-4232-8164-cc9d51af9ec6', --sjo heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta
            '586ab8c0-f7fa-4bb0-8eac-fde038f41ee2',-- SCL - Nunoa
            '55b113a6-baf1-4ab3-9e26-3117180de4af',--SCL - Lo Barnechea

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', -- RIO botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
             'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
             '812e6913-d78e-4dbc-877d-16993853aa7e', --sao domingos
             '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
             '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola

        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1
            )
            
, aov_fix as (

SELECT 
dc.country as country,
store_id,
AVG(subtotal) aov_fix

FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
LEFT JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
    
WHERE day_partition >= '2022-01-01'
AND (subtotal is not null or subtotal != 0)
AND NOT is_cancelled
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2022-01-01'
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
            'bc2ab3e9-e10d-420a-a3d0-395576fed377', -- lim la molina
            '993e277c-6d80-428c-8c8f-d2c9857eb8c1',  --LIM - San Borja
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
       
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
        -- MEDELLIN
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- SJO - Heredia
            '4364cfad-dd86-4232-8164-cc9d51af9ec6', --sjo heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta
            '586ab8c0-f7fa-4bb0-8eac-fde038f41ee2',-- SCL - Nunoa
             '55b113a6-baf1-4ab3-9e26-3117180de4af',--SCL - Lo Barnechea

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', -- RIO botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            'a63559d1-93c6-4f60-b683-1e207ccda10a' , -- SAO - Batista
            '812e6913-d78e-4dbc-877d-16993853aa7e', --sao domingos
            '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
            '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )
GROUP BY 1, 2

),

ordenes as (

SELECT 
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
--date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_week,
'LATAM' as region,
dc.country,
(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849',   -- LIM - Magdalena
         '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
         ' bc2ab3e9-e10d-420a-a3d0-395576fed377', --lim  la molina
         '993e277c-6d80-428c-8c8f-d2c9857eb8c1' --LIM - San Borja
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --BOG salitre
      )
   THEN
      'Bogota'
   WHEN 
      facility_id in 
      (
         '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' -- MED - DModa
      )
   THEN
       'Medellin'
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9',  -- SJO - Pavas
         '035c0131-50e4-465a-9596-ad8eb9be03f8', --sjo heredia
         '4364cfad-dd86-4232-8164-cc9d51af9ec6'-- SJO - Heredia
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee'   -- GDL - Av. Mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
         '3759ad21-4537-48ea-b1f1-48809e8a03ac' --rio botafogo
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         'a63559d1-93c6-4f60-b683-1e207ccda10a',  -- SAO - Batista
         '812e6913-d78e-4dbc-877d-16993853aa7e', -- sao domingos
         '14d9bb92-6376-4d69-adb6-64f97e4b3561', --sao ferreira
         '89cf86b9-9e74-4657-86df-2a851492af88' --sao espahola
      )
   then
      'Sao Paulo'
    WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- 'SCL - Recoleta'
             '55b113a6-baf1-4ab3-9e26-3117180de4af',--SCL - Lo Barnechea
            '586ab8c0-f7fa-4bb0-8eac-fde038f41ee2'-- SCL - Nunoa
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id= 'bc2ab3e9-e10d-420a-a3d0-395576fed377' THEN 'LIM - La Molina'
  WHEN facility_id= '993e277c-6d80-428c-8c8f-d2c9857eb8c1' THEN ' LIM - San Borja'

   
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
   
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
  WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
  WHEN facility_id = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   when facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id= '89cf86b9-9e74-4657-86df-2a851492af88' Then  'SAO - Guarulhos'
   WHEN facility_id= 'a63559d1-93c6-4f60-b683-1e207ccda10a'  THEN 'SAO - Batista'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
    when facility_id = '812e6913-d78e-4dbc-877d-16993853aa7e' then 'SAO - Domingos'
    WHEN facility_id = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   
   
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
  WHEN facility_id = '586ab8c0-f7fa-4bb0-8eac-fde038f41ee2' THEN 'SCL - Nunoa'
  WHEN facility_id = '55b113a6-baf1-4ab3-9e26-3117180de4af' THEN 'SCL - Lo Barnechea'

   
   WHEN facility_id = '4e36823e-98a2-4382-8797-48ef313196a9' THEN 'LIM - Lince'
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   WHEN facility_id = '035c0131-50e4-465a-9596-ad8eb9be03f8' THEN 'SJO - Heredia'
   WHEN facility_id = '4364cfad-dd86-4232-8164-cc9d51af9ec6' THEN 'SJO - Heredia'
  
  WHEN facility_id = '757182ff-7a54-4571-91d4-0027253c0117' THEN 'BOG - Salitre 2 EM'
   
   

   else co.facility_name
   
   
END
) AS facility_name,
organization_name,
organization_id,
tenant_type,
fo.first_order,
lo.last_order,

order_id,
is_cancelled,
subtotal,
ao.aov_fix,
usd_subtotal,
discount,
net_payout,
co.ofo_slug, co.external_order_id,co.order_items_quantity, co.day_partition,



(ROW_NUMBER() OVER (PARTITION BY co.ofo_slug, co.external_order_id,co.order_items_quantity, co.usd_subtotal ORDER BY co.order_id )) as row_number


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
LEFT JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
LEFT JOIN first_order fo 
   ON co.store_id=fo.store_id
LEFT JOIN last_order lo 
   ON co.store_id=lo.store_id
LEFT JOIN aov_fix ao 
   ON co.store_id=ao.store_id AND dc.country = ao.country
    
WHERE day_partition >= '2022-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2022-01-01'

AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
            '4e36823e-98a2-4382-8797-48ef313196a9', -- LIM - Lince
            'bc2ab3e9-e10d-420a-a3d0-395576fed377', -- LIM - La Molina
             '993e277c-6d80-428c-8c8f-d2c9857eb8c1', --LIM - San Borja
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117',  --'BOG - Salitre 2 EM'
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
        -- MEDELLIN
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f', -- MED - DModa
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
            '035c0131-50e4-465a-9596-ad8eb9be03f8', -- HEREDIA
            '4364cfad-dd86-4232-8164-cc9d51af9ec6',--heredia 2
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - Av. Mexico
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta
            '586ab8c0-f7fa-4bb0-8eac-fde038f41ee2',-- SCL - Nunoa
            '55b113a6-baf1-4ab3-9e26-3117180de4af',--SCL - Lo Barnechea

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
            '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac',--RIO - Botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53', -- SAO - Vargas
            '89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
            '812e6913-d78e-4dbc-877d-16993853aa7e',--SAO - Domingos
            '14d9bb92-6376-4d69-adb6-64f97e4b3561', --SAO - Ferreira
            '89cf86b9-9e74-4657-86df-2a851492af88', --SAO - Espanhola
            
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                )

)


select 
created_month,region, country, city,
facility_name,
organization_name,
organization_id,
tenant_type,
first_order,
last_order,




COUNT (DISTINCT order_id) AS all_orders,
COUNT (DISTINCT CASE WHEN is_cancelled THEN order_id END) AS cancelled_orders,
COUNT (DISTINCT order_id)-COUNT (DISTINCT CASE WHEN is_cancelled THEN order_id END) AS orders,
SUM(coalesce(subtotal, aov_fix)) AS all_gmv,
SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv,
SUM(coalesce(subtotal, 0)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS gmv,
(SUM(coalesce(subtotal, aov_fix)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, aov_fix) ELSE 0 END)) / (COUNT (DISTINCT order_id)-COUNT (DISTINCT case when is_cancelled  THEN order_id END)) AS aov,
SUM(usd_subtotal ) AS all_gmv_usd,
SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END) AS cancelled_gmv_usd,
SUM(usd_subtotal )-SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END) AS gmv_usd,
(SUM(usd_subtotal )-SUM(CASE WHEN is_cancelled THEN usd_subtotal ELSE 0 END)) / (COUNT (DISTINCT order_id)-COUNT (DISTINCT CASE WHEN is_cancelled = TRUE THEN order_id END)) AS aov_usd,
SUM(CASE WHEN NOT is_cancelled THEN coalesce(net_payout, 0) ELSE 0 END) AS net_payout,
SUM(CASE WHEN NOT is_cancelled THEN abs(discount) ELSE 0 END) AS discount,
(SUM(coalesce(subtotal, 0)) - SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END)) - SUM(CASE WHEN NOT is_cancelled THEN abs(discount) ELSE 0 END) AS net_gmv


from ordenes
where row_number=1

GROUP BY 1,2,3,4,5,6,7,8,9,10
ORDER BY 1 DESC,2 ,4,5,6,7


"""


print('Querying Latam Orders & GMV...')
latam = ds_presto.fetch_data(conn=conn, sql_str=query, use_cache=False)
print('Latam Orders & GMV Pulled ...')



Querying Latam Orders & GMV...
Latam Orders & GMV Pulled ...


In [5]:
# converting column numbers to numeric type
latam.aov = pd.to_numeric(latam.aov ,errors='coerce')


#If tenant type is null, return DELIVERY. 
latam.tenant_type = np.where(latam.tenant_type.isna(), 'DELIVERY', latam.tenant_type)

# If brand is null, return FALSE to the is new brand column. 
# latam.is_new_store = np.where(latam.is_new_store.isna(), False, latam.is_new_store)


In [6]:
latam

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,first_order,last_order,...,cancelled_gmv,gmv,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv
0,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Bendito Arroz 2,788fb991-5b69-4c28-bcae-68952f17f471,CLOUD_RETAIL,1.661207e+09,1.694632e+09,...,23900.00,928600.00,22109.523810,232.967754,5.845595,227.122159,5.40767,0.0000,39900.00,888700.00
1,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Deli reposteria,e3a0f31a-0d4a-4a41-a3a1-e83725e3ce2a,CLOUD_RETAIL,1.668621e+09,1.694633e+09,...,299700.00,3342900.00,29583.185841,890.927394,73.302295,817.625099,7.23562,0.0000,171800.00,3171100.00
2,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Frambas,71db16c6-f1ad-4c76-8e6f-17f55337635a,CLOUD_RETAIL,1.659126e+09,1.694478e+09,...,0.00,266000.00,38000.000000,65.059761,0.000000,65.059761,9.294252,0.0000,NaN,NaN
3,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Franui,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,CLOUD_RETAIL,1.651533e+09,1.694553e+09,...,55800.00,920700.00,34100.000000,238.837808,13.647875,225.189933,8.340368,0.0000,20900.00,899800.00
4,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Heladeria Magda,e218a8a1-c89a-44d8-b504-ce2622d562e1,CLOUD_RETAIL,1.650741e+09,1.694547e+09,...,0.00,1207000.00,50291.666667,295.214782,0.000000,295.214782,12.300616,0.0000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37060,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,SP Chicken - KC,c553ea99-726b-4793-8309-7c88ccbf098e,DELIVERY,1.635291e+09,1.646090e+09,...,563.50,24921.80,47.289943,4575.061025,101.143247,4473.917778,8.489408,17982.4113,3925.00,20996.80
37061,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,SP Chicken - KC,c553ea99-726b-4793-8309-7c88ccbf098e,DELIVERY,1.638485e+09,1.646089e+09,...,25.00,758.70,58.361538,140.667192,4.487278,136.179914,10.475378,0.0000,0.00,758.70
37062,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,Senhor Pudim - KC,f4da1681-ce95-4bae-8814-31dc508f4a9d,VIRTUAL,1.634224e+09,1.643590e+09,...,13.86,4291.68,62.198261,772.806203,2.487747,770.318456,11.164036,2926.9751,5.99,4285.69
37063,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,WineBar - KC,15e7b7ea-e8f8-4439-834b-f56edfa8c590,VIRTUAL,1.627853e+09,1.646242e+09,...,0.00,239.00,79.666667,42.898378,0.000000,42.898378,14.299459,162.0350,13.98,225.02


In [7]:
primera_orden=[]
for i in latam['first_order']:
    try:
        a=(datetime.fromtimestamp(i))
    except:
        a=(i)
    primera_orden.append(a)

In [8]:
latam['first_order_Date']=primera_orden

In [9]:
ultima_orden=[]
for i in latam['last_order']:
    try:
        a=(datetime.fromtimestamp(i))
    except:
        a=(i)
    ultima_orden.append(a)

In [10]:
latam['last_order_Date']=ultima_orden

In [11]:
latam

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,first_order,last_order,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Bendito Arroz 2,788fb991-5b69-4c28-bcae-68952f17f471,CLOUD_RETAIL,1.661207e+09,1.694632e+09,...,22109.523810,232.967754,5.845595,227.122159,5.40767,0.0000,39900.00,888700.00,2022-08-22 22:29:52,2023-09-13 19:09:28
1,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Deli reposteria,e3a0f31a-0d4a-4a41-a3a1-e83725e3ce2a,CLOUD_RETAIL,1.668621e+09,1.694633e+09,...,29583.185841,890.927394,73.302295,817.625099,7.23562,0.0000,171800.00,3171100.00,2022-11-16 17:51:17,2023-09-13 19:30:49
2,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Frambas,71db16c6-f1ad-4c76-8e6f-17f55337635a,CLOUD_RETAIL,1.659126e+09,1.694478e+09,...,38000.000000,65.059761,0.000000,65.059761,9.294252,0.0000,NaN,NaN,2022-07-29 20:11:44,2023-09-12 00:23:36
3,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Franui,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,CLOUD_RETAIL,1.651533e+09,1.694553e+09,...,34100.000000,238.837808,13.647875,225.189933,8.340368,0.0000,20900.00,899800.00,2022-05-02 23:10:51,2023-09-12 21:02:07
4,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Heladeria Magda,e218a8a1-c89a-44d8-b504-ce2622d562e1,CLOUD_RETAIL,1.650741e+09,1.694547e+09,...,50291.666667,295.214782,0.000000,295.214782,12.300616,0.0000,NaN,NaN,2022-04-23 19:16:36,2023-09-12 19:25:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37060,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,SP Chicken - KC,c553ea99-726b-4793-8309-7c88ccbf098e,DELIVERY,1.635291e+09,1.646090e+09,...,47.289943,4575.061025,101.143247,4473.917778,8.489408,17982.4113,3925.00,20996.80,2021-10-26 23:34:44,2022-02-28 23:19:13
37061,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,SP Chicken - KC,c553ea99-726b-4793-8309-7c88ccbf098e,DELIVERY,1.638485e+09,1.646089e+09,...,58.361538,140.667192,4.487278,136.179914,10.475378,0.0000,0.00,758.70,2021-12-02 22:40:22,2022-02-28 22:50:59
37062,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,Senhor Pudim - KC,f4da1681-ce95-4bae-8814-31dc508f4a9d,VIRTUAL,1.634224e+09,1.643590e+09,...,62.198261,772.806203,2.487747,770.318456,11.164036,2926.9751,5.99,4285.69,2021-10-14 15:12:56,2022-01-31 00:54:36
37063,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,WineBar - KC,15e7b7ea-e8f8-4439-834b-f56edfa8c590,VIRTUAL,1.627853e+09,1.646242e+09,...,79.666667,42.898378,0.000000,42.898378,14.299459,162.0350,13.98,225.02,2021-08-01 21:31:09,2022-03-02 17:34:51


In [12]:
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#

In [13]:
#g_sheets_creds_path = '/home/ruben.lopez/creds_spread.json'

In [14]:
## Getting all lighttech information from BRA

## GSheets Oauth
gc = pygsheets.authorize(custom_credentials=credentials_sheets)

############# SÃO PAULO ############################################

# ACRE
acre = gc.open_by_key('1u8N3EHnRxpRB3o12kkk89qCXMLYV_erAHJS9nvyx_jM')
wsacre = acre.worksheet('title','Feed Runner')
acredf = wsacre.get_as_df(has_header=True,start="B8", end=None,empty_value='')


In [15]:
# CLELIA
clelia = gc.open_by_key('1Co1FEKw_TOTE1FFp_WBBvQg3wQt3w256_rD0nCGjbJY')
wsclelia = clelia.worksheet('title','Feed Runner')
cleliadf = wsclelia.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# DINO
dino = gc.open_by_key('1x9rbysjVVL0Y8LDEkQioWgG-CmpEfovWMsBfD33CZXU')
wsdino = dino.worksheet('title','Feed Runner')
dinodf = wsdino.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# GUARARAPES
guarara = gc.open_by_key('1QT4hdgZe4lwE19eo8yQ6LS6Rv2ozvOycjAy6DkjBTn4')
wsguarara = guarara.worksheet('title','Feed Runner')
guararadf = wsguarara.get_as_df(has_header=True,start="B8", end=None,empty_value='')



/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pygsheets/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pygsheets/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify

In [16]:
# BORBA
#borba = gc.open_by_key('1an2a2bnzJ_iqenuMiCQvEXdUcJMz8KwaEL21OriZYW8')
#wsborba = borba.worksheet('title','Feed Runner')
#borbadf = wsborba.get_as_df(has_header=True,start="B8", end=None,empty_value='')


############# BELO HORIZONTE ############################################

# INDUSTRIAL
#industrial = gc.open_by_key('1My9Ph_c0nM7eYASIIjcYy1cFw8Ip_T4rhOUgnM73O0M')
#wsindustrial = industrial.worksheet('title','Repor')
#industrialdf = wsindustrial.get_as_df(has_header=True,start="A1", end=None,empty_value='')

# ARTHUR GUIMARÃES
ag = gc.open_by_key('1wC3GcubNMLIa4pDT0HxRfdfucQZo5PCHXzifwLZHNgE')
wsag = ag.worksheet('title','Feed Runner')
agdf = wsag.get_as_df(has_header=True,start="B8", end=None,empty_value='')


# PARACATU
paracatu = gc.open_by_key('1EVuoGK-EAixIFXYu8jLdjEgZpIdkzR_5AeB55PLMsME')
wsparacatu = paracatu.worksheet('title','Feed Runner')
paracatudf = wsparacatu.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# CASTELO
castelo = gc.open_by_key('1gkbjx3nHJuDNd1QaJu0GwrSz4oe6qsKulnFIQmc1PSo')
wscastelo = castelo.worksheet('title','Feed Runner')
castelodf = wscastelo.get_as_df(has_header=True,start="B8", end=None,empty_value='')


############# RIO DE JANEIRO ############################################


# MEIER
meier = gc.open_by_key('1deVBnnKTjU5zhxj2S3PGPaTmbrPfHmvF08U6_8BZm9Y')
wsmeier = meier.worksheet('title','Feed Runner')
meierdf = wsmeier.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# BARRA
barra = gc.open_by_key('10ODauBi60mpfxiqIQ76vSxR2JJNmBnkxzCSzYGHufWI')
wsbarra = barra.worksheet('title','Feed Runner')
barradf = wsbarra.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# CAMERINO
camerino = gc.open_by_key('1KP6d_NKPMfYafW1b_i62O1PIx6n0zJUW-mSUWf468XU')
wscamerino = camerino.worksheet('title','Feed Runner')
camerinodf = wscamerino.get_as_df(has_header=True,start="B8", end=None,empty_value='')


# NITEROI
niteroi = gc.open_by_key('1jOlzMt4a6rBYcgIpwel5uZTFOE6XZBriTJnDxsLBi1g')
wsniteroi = niteroi.worksheet('title','Feed Runner')
niteroidf = wsniteroi.get_as_df(has_header=True,start="B8", end=None,empty_value='')



# GUARULHOS
guaru_hist = gc.open_by_key('1Rbl7xZazZgmsF9nW4bWnjXrz86zr94nbCkOftIl2bMw')
wsguaru_hist = guaru_hist.worksheet('title','Feed Runner')
guarudf_hist = wsguaru_hist.get_as_df(has_header=True,start="B8", end=None,empty_value='')

# TIJUCA
tijuca_hist = gc.open_by_key('1xIlU8LWuhYMI4X9yDvT7K060QU89RbSSkLnIQ5wT2oY')
wstijuca_hist = tijuca_hist.worksheet('title','Feed Runner')
tijucadf_hist = wstijuca_hist.get_as_df(has_header=True,start="B8", end=None,empty_value='')







/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pygsheets/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pygsheets/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify

In [17]:



# dataframes to list
dataframes = [acredf, cleliadf, dinodf, guararadf, #industrialdf, 
              agdf, 
              paracatudf, meierdf, barradf, #borbadf, 
              camerinodf, castelodf,
              niteroidf,guarudf_hist,tijucadf_hist]

In [18]:
niteroidf.head()

,Número \ndo pedido,Restaurante,Organization,Aplicativo,Valor do pedido\n(use vírgula),Facility,Hora de Registro,Número pedido,
0,NITEROI9,American Burger - KC,American Burger - KC,UberEats,"R$ 37,97",SA-BR-RJ-Niteroi,2021-10-27 22:18:40,077F7,
1,NITEROI10,American Burger - KC,American Burger - KC,UberEats,"R$ 7,99",SA-BR-RJ-Niteroi,2021-10-27 22:20:05,18DC6,
2,NITEROI11,American Burger - KC,American Burger - KC,UberEats,"R$ 8,99",SA-BR-RJ-Niteroi,2021-10-27 22:21:49,08EB1,
3,NITEROI12,American Burger - KC,American Burger - KC,UberEats,"R$ 7,99",SA-BR-RJ-Niteroi,2021-10-27 22:49:32,E5FOA,
4,NITEROI13,American Burger - KC,American Burger - KC,UberEats,"R$ 7,99",SA-BR-RJ-Niteroi,2021-10-27 23:37:55,74B18,


In [19]:
# concatenating lighttech DataFrames
lighttech = pd.concat(dataframes)

In [20]:
# renaming columns to match Presto Query (latam df) headers
lighttech.rename(columns={'Número \ndo pedido':'orders', 
                         'Organization':'organization_name',
                         'Valor do pedido\n(use vírgula)':'gmv',
                         'Facility':'facility_name',
                         'Hora de Registro':'created_date'}, inplace=True)

In [21]:
# eliminating unused columns
lighttech = lighttech[['created_date', 'orders', 'facility_name', 'organization_name', 'gmv']]

# replacing whitespaces with NaN
lighttech = lighttech.replace(r'^\s*$', np.nan, regex=True)

# cleaning dates
lighttech['created_date'] = lighttech['created_date'].str.replace('/', '-').str.replace('--', '-')

# converting to datetime
lighttech['created_date'] = pd.to_datetime(lighttech['created_date'], errors = 'coerce')

# eliminating rows without date or facility name
lighttech.dropna(subset=['created_date', 'facility_name'], inplace=True)

# cleaning the order amout column. Removing the R$ and replacing comma with dot. 
lighttech['gmv'] = lighttech['gmv'].astype(str).apply(lambda x: x.replace("R$","").replace(",", "."))

# converting to numeric type the gmv column. All errors will be set as NAN
lighttech['gmv'] = pd.to_numeric(lighttech['gmv'] ,errors='coerce')


# removing the time part from the timestamp
lighttech.created_date = lighttech.created_date.dt.date
lighttech.created_date = lighttech.created_date.astype('datetime64[ns]')


# getting first day of week
lighttech['dayofweek'] = lighttech.created_date.dt.weekday
lighttech['created_week'] = (lighttech.created_date - lighttech.dayofweek * timedelta(days=1))

# getting first day of month
lighttech['created_month'] = lighttech.created_date.dt.to_period('M').dt.to_timestamp()

/tmp/ipykernel_3610/3589416785.py:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  lighttech['created_date'] = pd.to_datetime(lighttech['created_date'], errors = 'coerce')


In [22]:
lighttech

,created_date,orders,facility_name,organization_name,gmv,dayofweek,created_week,created_month
0,2020-12-22,ACRE9,BR - Sao Paulo - Acre,American Burger Mooca,10.50,1,2020-12-21,2020-12-01
1,2020-12-22,ACRE10,BR - Sao Paulo - Acre,American Burger Mooca,7.51,1,2020-12-21,2020-12-01
2,2020-12-22,ACRE11,BR - Sao Paulo - Acre,American Burger Mooca,14.51,1,2020-12-21,2020-12-01
3,2020-12-22,ACRE12,BR - Sao Paulo - Acre,American Burger Mooca,31.51,1,2020-12-21,2020-12-01
4,2020-12-22,ACRE13,BR - Sao Paulo - Acre,American Burger Mooca,22.50,1,2020-12-21,2020-12-01
...,...,...,...,...,...,...,...,...
225,2022-04-11,NITEROI234,SA-BR-RJ-Niteroi,Baked potato,29.90,0,2022-04-11,2022-04-01
226,2022-04-17,NITEROI235,SA-BR-RJ-Niteroi,Pizzaria Aham,73.00,6,2022-04-11,2022-04-01
227,2022-04-17,NITEROI236,SA-BR-RJ-Niteroi,Pizzaria Aham,57.00,6,2022-04-11,2022-04-01
228,2022-09-24,NITEROI237,SA-BR-RJ-Niteroi,Pizzaria Aham,49.00,5,2022-09-19,2022-09-01


In [23]:
# getting exchange rate
#path_cred = '/home/ruben.lopez/creds_spread.json'
gc = pygsheets.authorize(custom_credentials=credentials_sheets)
dl = gc.open_by_key('1h_lgOO2wf89txM2jX5ZXS64DizqewyPUReI_0-zP2M0')
dlx = dl.worksheet('title','latam_monthly')
dldf = dlx.get_as_df(has_header=True,start="A1", end=None,empty_value='')
dldf.rename(columns={'exchange_rate':'USD'}, inplace=True)

In [24]:
dldf

,created_month,country,currency_code,USD
0,2019-01-01,Mexico,MXN,18.8
1,2019-02-01,Mexico,MXN,19.59
2,2019-03-01,Mexico,MXN,22.26
3,2019-04-01,Mexico,MXN,24.2
4,2019-05-01,Mexico,MXN,23.47
...,...,...,...,...
679,2024-12-01,USA,USD,#N/A
680,2025-01-01,USA,USD,#N/A
681,2025-02-01,USA,USD,#N/A
682,2025-03-01,USA,USD,#N/A


In [25]:
# preparing the merge and merging
dldf['Date'] = pd.to_datetime(dldf['created_month'])

lighttech = pd.merge(lighttech, dldf, left_on="created_date", right_on="Date", how="left",suffixes=('', '_cambio') )

lighttech['gmv_usd'] = lighttech.gmv / lighttech.USD
# lighttech['aov_usd'] = lighttech.gmv_usd / lighttech.orders
# lighttech.orders = pd.to_numeric(lighttech.orders)

In [26]:
lighttech 

,created_date,orders,facility_name,organization_name,gmv,dayofweek,created_week,created_month,created_month_cambio,country,currency_code,USD,Date,gmv_usd
0,2020-12-22,ACRE9,BR - Sao Paulo - Acre,American Burger Mooca,10.50,1,2020-12-21,2020-12-01,NaN,NaN,NaN,NaN,NaT,NaN
1,2020-12-22,ACRE10,BR - Sao Paulo - Acre,American Burger Mooca,7.51,1,2020-12-21,2020-12-01,NaN,NaN,NaN,NaN,NaT,NaN
2,2020-12-22,ACRE11,BR - Sao Paulo - Acre,American Burger Mooca,14.51,1,2020-12-21,2020-12-01,NaN,NaN,NaN,NaN,NaT,NaN
3,2020-12-22,ACRE12,BR - Sao Paulo - Acre,American Burger Mooca,31.51,1,2020-12-21,2020-12-01,NaN,NaN,NaN,NaN,NaT,NaN
4,2020-12-22,ACRE13,BR - Sao Paulo - Acre,American Burger Mooca,22.50,1,2020-12-21,2020-12-01,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147895,2022-04-11,NITEROI234,SA-BR-RJ-Niteroi,Baked potato,29.90,0,2022-04-11,2022-04-01,NaN,NaN,NaN,NaN,NaT,NaN
147896,2022-04-17,NITEROI235,SA-BR-RJ-Niteroi,Pizzaria Aham,73.00,6,2022-04-11,2022-04-01,NaN,NaN,NaN,NaN,NaT,NaN
147897,2022-04-17,NITEROI236,SA-BR-RJ-Niteroi,Pizzaria Aham,57.00,6,2022-04-11,2022-04-01,NaN,NaN,NaN,NaN,NaT,NaN
147898,2022-09-24,NITEROI237,SA-BR-RJ-Niteroi,Pizzaria Aham,49.00,5,2022-09-19,2022-09-01,NaN,NaN,NaN,NaN,NaT,NaN


In [27]:
lighttech_mes=lighttech.groupby(['created_month',
                      'facility_name',
                      'organization_name']
                     , as_index=False
                     , dropna=False
                     ).agg({'orders': 'count',
                            'gmv': 'sum', 
                           'gmv_usd': 'sum'}).reset_index(drop=True)

In [28]:
lighttech_mes

,created_month,facility_name,organization_name,orders,gmv,gmv_usd
0,2020-12-01,BR - Sao Paulo - Acre,American Burger Mooca,16,376.31,0
1,2020-12-01,BR - Sao Paulo - Acre,Galeto's KC,320,14571.50,0
2,2020-12-01,BR - Sao Paulo - Acre,Papila,1482,83209.15,0
3,2020-12-01,BR - Sao Paulo - Acre,Vinil Burger,21,1618.10,0
4,2020-12-01,BR - Sao Paulo - Clélia,Tradi Hamburgueria,2999,188829.77,0
...,...,...,...,...,...,...
361,2023-05-01,BR - Sao Paulo - Guararapes,Jurassic Park - KC,80,5528.63,0
362,2023-05-01,BR - Sao Paulo - Guararapes,fat buda,1,617.00,0
363,2023-06-01,BR - Sao Paulo - Guararapes,Jurassic Park - KC,59,5936.30,0
364,2023-07-01,BR - Sao Paulo - Guararapes,Jurassic Park - KC,31,2990.95,755.830098


In [29]:
lighttech_mes.orders = pd.to_numeric(lighttech_mes.orders)

# creating new columns to match the Presto query headers
lighttech_mes['region'] = 'LATAM'
lighttech_mes['country'] = 'Brazil'
lighttech_mes['tenant_type'] = 'DELIVERY'
# lighttech['is_new_store'] = False
lighttech_mes['all_orders'] = lighttech_mes.orders
lighttech_mes['cancelled_orders'] = 0
lighttech_mes['all_gmv'] = lighttech_mes.gmv
lighttech_mes['cancelled_gmv'] = 0
lighttech_mes['aov'] = lighttech_mes.gmv / lighttech_mes.orders
lighttech_mes['all_gmv_usd'] = np.nan
lighttech_mes['cancelled_gmv_usd'] = np.nan
lighttech_mes['aov_usd'] = lighttech_mes.gmv_usd / lighttech_mes.orders


In [30]:
lighttech_mes['facility_name'] = lighttech_mes['facility_name'].apply(facility)
lighttech_mes['city'] = lighttech_mes['facility_name'].apply(city)
lighttech_mes['organization_id'] = np.nan
lighttech_mes['net_payout'] = lighttech_mes.gmv
lighttech_mes['net_gmv'] = lighttech_mes.gmv
lighttech_mes['discount'] = 0
lighttech_mes['first_order_Date']=np.nan
lighttech_mes['last_order_Date']=np.nan



In [31]:
latam_2=latam[['created_month', 'region', 'country', 'city', 'facility_name',
       'organization_name', 'organization_id', 'tenant_type',
       'all_orders', 'cancelled_orders', 'orders', 'all_gmv', 'cancelled_gmv',
       'gmv', 'aov', 'all_gmv_usd', 'cancelled_gmv_usd', 'gmv_usd', 'aov_usd',
       'net_payout', 'discount', 'net_gmv', 'first_order_Date','last_order_Date']]

In [32]:
latam_2

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Bendito Arroz 2,788fb991-5b69-4c28-bcae-68952f17f471,CLOUD_RETAIL,43,1,...,22109.523810,232.967754,5.845595,227.122159,5.40767,0.0000,39900.00,888700.00,2022-08-22 22:29:52,2023-09-13 19:09:28
1,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Deli reposteria,e3a0f31a-0d4a-4a41-a3a1-e83725e3ce2a,CLOUD_RETAIL,118,5,...,29583.185841,890.927394,73.302295,817.625099,7.23562,0.0000,171800.00,3171100.00,2022-11-16 17:51:17,2023-09-13 19:30:49
2,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Frambas,71db16c6-f1ad-4c76-8e6f-17f55337635a,CLOUD_RETAIL,7,0,...,38000.000000,65.059761,0.000000,65.059761,9.294252,0.0000,NaN,NaN,2022-07-29 20:11:44,2023-09-12 00:23:36
3,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Franui,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,CLOUD_RETAIL,28,1,...,34100.000000,238.837808,13.647875,225.189933,8.340368,0.0000,20900.00,899800.00,2022-05-02 23:10:51,2023-09-12 21:02:07
4,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Heladeria Magda,e218a8a1-c89a-44d8-b504-ce2622d562e1,CLOUD_RETAIL,24,0,...,50291.666667,295.214782,0.000000,295.214782,12.300616,0.0000,NaN,NaN,2022-04-23 19:16:36,2023-09-12 19:25:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37060,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,SP Chicken - KC,c553ea99-726b-4793-8309-7c88ccbf098e,DELIVERY,539,12,...,47.289943,4575.061025,101.143247,4473.917778,8.489408,17982.4113,3925.00,20996.80,2021-10-26 23:34:44,2022-02-28 23:19:13
37061,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,SP Chicken - KC,c553ea99-726b-4793-8309-7c88ccbf098e,DELIVERY,14,1,...,58.361538,140.667192,4.487278,136.179914,10.475378,0.0000,0.00,758.70,2021-12-02 22:40:22,2022-02-28 22:50:59
37062,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,Senhor Pudim - KC,f4da1681-ce95-4bae-8814-31dc508f4a9d,VIRTUAL,70,1,...,62.198261,772.806203,2.487747,770.318456,11.164036,2926.9751,5.99,4285.69,2021-10-14 15:12:56,2022-01-31 00:54:36
37063,2022-01-01,LATAM,Brazil,Sao Paulo,SAO - Vargas,WineBar - KC,15e7b7ea-e8f8-4439-834b-f56edfa8c590,VIRTUAL,3,0,...,79.666667,42.898378,0.000000,42.898378,14.299459,162.0350,13.98,225.02,2021-08-01 21:31:09,2022-03-02 17:34:51


In [33]:
# ordering the columns to match the Presto query result
lighttech_merge = lighttech_mes[[c for c in latam_2.columns]]

In [34]:
lighttech_merge

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Rua do Acre,American Burger Mooca,NaN,DELIVERY,16,0,...,23.519375,NaN,NaN,0,0.0,376.31,0,376.31,NaN,NaN
1,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Rua do Acre,Galeto's KC,NaN,DELIVERY,320,0,...,45.535938,NaN,NaN,0,0.0,14571.50,0,14571.50,NaN,NaN
2,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Rua do Acre,Papila,NaN,DELIVERY,1482,0,...,56.146525,NaN,NaN,0,0.0,83209.15,0,83209.15,NaN,NaN
3,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Rua do Acre,Vinil Burger,NaN,DELIVERY,21,0,...,77.052381,NaN,NaN,0,0.0,1618.10,0,1618.10,NaN,NaN
4,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Lapa Clelia,Tradi Hamburgueria,NaN,DELIVERY,2999,0,...,62.964245,NaN,NaN,0,0.0,188829.77,0,188829.77,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2023-05-01,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),Jurassic Park - KC,NaN,DELIVERY,80,0,...,69.107875,NaN,NaN,0,0.0,5528.63,0,5528.63,NaN,NaN
362,2023-05-01,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),fat buda,NaN,DELIVERY,1,0,...,617.000000,NaN,NaN,0,0.0,617.00,0,617.00,NaN,NaN
363,2023-06-01,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),Jurassic Park - KC,NaN,DELIVERY,59,0,...,100.615254,NaN,NaN,0,0.0,5936.30,0,5936.30,NaN,NaN
364,2023-07-01,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),Jurassic Park - KC,NaN,DELIVERY,31,0,...,96.482258,NaN,NaN,755.830098,24.381616,2990.95,0,2990.95,NaN,NaN


In [35]:
############# RAPPI

# RAPPI
rappi = gc.open_by_key('1XfvvARTR0-fNvgcy8UnXxfTWIxFifCF5XQpJW5lRctE')
wsrappi = rappi.worksheet('title','Data - Rappi Shopper Monthly')
rappidf = wsrappi.get_as_df(has_header=True,start="A", end="AD",empty_value='')

rappidf.rename(columns={'month':'created_month', 
                         'week':'created_week',
                         'organization':'organization_name',
                         # 'Facility':'facility_name',
                         'PRECIO TOTAL':'gmv',
                         'ID ORDEN':'orders'}, inplace=True)

rappi_status = ['Finalizado', 'pending_review', 'finished']

rappidf = rappidf[rappidf.ESTADO.isin(rappi_status)]

# eliminating unused columns
rappidf = rappidf[['created_month', 'created_week', 'facility_name', 'organization_name', 'gmv', 'orders']]


# converting to numeric type the gmv column. All errors will be set as NAN
rappidf['gmv'] = pd.to_numeric(rappidf['gmv'] ,errors='coerce')


rappidf.created_month = pd.to_datetime(rappidf.created_month, errors='coerce')
rappidf.created_week = pd.to_datetime(rappidf.created_week, errors='coerce')


rappidf = rappidf.groupby(['created_month',
                      'facility_name',
                      'organization_name']
                     , as_index=False
                     , dropna=False
                     ).agg({'orders': 'nunique',
                            'gmv': 'sum'}).reset_index(drop=True)


rappidf['region'] = 'LATAM'
rappidf['country'] = 'Colombia'
rappidf['tenant_type'] = 'VIRTUAL'
# rappidf['is_new_store'] = False
rappidf['all_orders'] = rappidf.orders
rappidf['cancelled_orders'] = 0
rappidf['all_gmv'] = rappidf.gmv
rappidf['cancelled_gmv'] = 0
rappidf['aov'] = rappidf.gmv / rappidf.orders
rappidf['all_gmv_usd'] = np.nan
rappidf['gmv_usd'] = np.nan
rappidf['cancelled_gmv_usd'] = np.nan
rappidf['aov_usd'] = np.nan
rappidf['facility_name'] = rappidf['facility_name'].apply(facility)
rappidf['total'] = rappidf.gmv
rappidf['organization_id'] = np.nan
rappidf['net_payout'] = rappidf.gmv
rappidf['net_gmv'] = rappidf.gmv
rappidf['discount'] = 0
rappidf['first_order_Date']=np.nan
rappidf['last_order_Date']=np.nan
# eliminating rows without date or facility name
rappidf.dropna(subset=['facility_name', 'created_month'], inplace=True)

rappidf['city'] = rappidf['facility_name'].apply(city)


# ordering the columns to match the Presto query result
rappidf = rappidf[[c for c in latam_2.columns]]


In [36]:
rappidf

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2020-11-01,LATAM,Colombia,Bogota,BOG - Atabanza,El Pomar,NaN,VIRTUAL,6,0,...,20183.333333,NaN,NaN,NaN,NaN,121100,0,121100,NaN,NaN
1,2020-11-01,LATAM,Colombia,Bogota,BOG - Castellana,Atlantic,NaN,VIRTUAL,216,0,...,91854.629630,NaN,NaN,NaN,NaN,19840600,0,19840600,NaN,NaN
2,2020-11-01,LATAM,Colombia,Bogota,BOG - Castellana,El Pomar,NaN,VIRTUAL,15,0,...,33556.666667,NaN,NaN,NaN,NaN,503350,0,503350,NaN,NaN
3,2020-11-01,LATAM,Colombia,Bogota,BOG - Castellana,Sannus Foods,NaN,VIRTUAL,142,0,...,79491.112676,NaN,NaN,NaN,NaN,11287738,0,11287738,NaN,NaN
4,2020-11-01,LATAM,Colombia,Bogota,BOG - Puente Aranda,El Pomar,NaN,VIRTUAL,6,0,...,31616.666667,NaN,NaN,NaN,NaN,189700,0,189700,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,2023-05-01,LATAM,Colombia,Medellin,MED - DModa,Lauval Floral,NaN,VIRTUAL,57,0,...,47035.087719,NaN,NaN,NaN,NaN,2681000,0,2681000,NaN,NaN
267,2023-05-01,LATAM,Colombia,Bogota,BOG - San Felipe,Food BARF,NaN,VIRTUAL,23,0,...,26982.608696,NaN,NaN,NaN,NaN,620600,0,620600,NaN,NaN
268,2023-05-01,LATAM,Colombia,Bogota,BOG - Atabanza,Food BARF,NaN,VIRTUAL,5,0,...,32970.000000,NaN,NaN,NaN,NaN,164850,0,164850,NaN,NaN
269,2023-05-01,LATAM,Colombia,Cali,CLO - Chipichape,Lauval Floral,NaN,VIRTUAL,79,0,...,56924.050633,NaN,NaN,NaN,NaN,4497000,0,4497000,NaN,NaN


In [37]:
# dfs = [latam, lighttech]
# concating the 
#final = pd.concat([latam_2, lighttech_mes, rappidf], ignore_index=True)
final = pd.concat([latam_2, lighttech_mes], ignore_index=True)

In [38]:
final

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Bendito Arroz 2,788fb991-5b69-4c28-bcae-68952f17f471,CLOUD_RETAIL,43,1,...,22109.523810,232.967754,5.845595,227.122159,5.40767,0.00,39900.0,888700.00,2022-08-22 22:29:52,2023-09-13 19:09:28
1,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Deli reposteria,e3a0f31a-0d4a-4a41-a3a1-e83725e3ce2a,CLOUD_RETAIL,118,5,...,29583.185841,890.927394,73.302295,817.625099,7.23562,0.00,171800.0,3171100.00,2022-11-16 17:51:17,2023-09-13 19:30:49
2,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Frambas,71db16c6-f1ad-4c76-8e6f-17f55337635a,CLOUD_RETAIL,7,0,...,38000.000000,65.059761,0.000000,65.059761,9.294252,0.00,NaN,NaN,2022-07-29 20:11:44,2023-09-12 00:23:36
3,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Franui,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,CLOUD_RETAIL,28,1,...,34100.000000,238.837808,13.647875,225.189933,8.340368,0.00,20900.0,899800.00,2022-05-02 23:10:51,2023-09-12 21:02:07
4,2023-09-01,LATAM,Colombia,Barranquilla,BAQ - Alto Prado,Heladeria Magda,e218a8a1-c89a-44d8-b504-ce2622d562e1,CLOUD_RETAIL,24,0,...,50291.666667,295.214782,0.000000,295.214782,12.300616,0.00,NaN,NaN,2022-04-23 19:16:36,2023-09-12 19:25:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-05-01 00:00:00,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),Jurassic Park - KC,NaN,DELIVERY,80,0,...,69.107875,NaN,NaN,0,0.0,5528.63,0.0,5528.63,NaT,NaT
37427,2023-05-01 00:00:00,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),fat buda,NaN,DELIVERY,1,0,...,617.000000,NaN,NaN,0,0.0,617.00,0.0,617.00,NaT,NaT
37428,2023-06-01 00:00:00,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),Jurassic Park - KC,NaN,DELIVERY,59,0,...,100.615254,NaN,NaN,0,0.0,5936.30,0.0,5936.30,NaT,NaT
37429,2023-07-01 00:00:00,LATAM,Brazil,Sao Paulo,SAO - Guararapes (Brooklin),Jurassic Park - KC,NaN,DELIVERY,31,0,...,96.482258,NaN,NaN,755.830098,24.381616,2990.95,0.0,2990.95,NaT,NaT


In [39]:
final_exp=final.groupby(['created_month','region', 'country', 'city',
                      'facility_name','first_order_Date','last_order_Date',
                      'organization_name', 'organization_id','tenant_type']
                     , as_index=False
                     , dropna=False
                     ).sum().reset_index(drop=True)

/tmp/ipykernel_3610/3395571886.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_exp=final.groupby(['created_month','region', 'country', 'city',
/tmp/ipykernel_3610/3395571886.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  final_exp=final.groupby(['created_month','region', 'country', 'city',


In [40]:
final_exp

,created_month,region,country,city,facility_name,first_order_Date,last_order_Date,organization_name,organization_id,tenant_type,...,orders,all_gmv,cancelled_gmv,gmv,aov,all_gmv_usd,cancelled_gmv_usd,net_payout,discount,net_gmv
0,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,NaT,NaT,Basilico Pizza - KC,NaN,DELIVERY,...,11,369.91,0.00,369.91,33.628182,0.000000,0.000000,369.91,0.00,369.91
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,NaT,NaT,Philippes Pizza - KC,NaN,DELIVERY,...,70,4027.10,0.00,4027.10,57.530000,0.000000,0.000000,4027.10,0.00,4027.10
2,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,NaT,NaT,Saú Culinária Saudável,NaN,DELIVERY,...,11,445.60,0.00,445.60,40.509091,0.000000,0.000000,445.60,0.00,445.60
3,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,1,74.80,0.00,74.80,74.800000,0.000000,0.000000,74.80,0.00,74.80
4,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Dino,NaT,NaT,American Burger,NaN,DELIVERY,...,679,22590.02,0.00,22590.02,33.269543,0.000000,0.000000,22590.02,0.00,22590.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-06-15 21:30:02,NaT,Palermo - CK,b238f8ff-e3ac-4c62-a940-f6eac0ed5a57,DELIVERY,...,171,7887.50,28.50,7859.00,45.959064,2126.182617,7.682562,0.00,262.20,7596.80
37427,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-06-24 19:10:19,NaT,MEGABITE,e27bf782-a8b4-46e8-8643-c5ea5e6cdfdd,DELIVERY,...,63,3071.61,45.53,3026.08,48.033016,828.191113,12.273229,0.00,553.61,2472.47
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-08-23 19:08:37,NaT,Pikalo - CK,6a8d7b6c-7de9-4c07-a3e2-a4f6b630b2dc,DELIVERY,...,0,396.50,396.50,0.00,0.000000,107.231452,107.231452,0.00,0.00,0.00
37429,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-09-08 20:16:09,NaT,Corporacion Baguetera SAC,ad0cc4ed-50d2-4489-a6bb-a463dbba000d,DELIVERY,...,17,832.00,0.00,832.00,48.941176,224.276886,0.000000,0.00,0.00,0.00


In [41]:
final_exp['facility_name'].value_counts()

MEX - Polanco                  1800
MEX - Pedregal                 1674
MEX - Roma                     1599
SAO - Guararapes (Brooklin)    1542
LIM - Magdalena                1426
SAO - Rua do Acre              1414
SAO - Dino                     1387
MEX - Napoles                  1370
BOG - Salitre 2 EM             1141
MEX - Santa Fe                 1133
BHZ - Paracatu                 1079
SAO - Lapa Clelia               983
BOG - Atabanza                  970
GDL - Av. Mexico                951
RIO - Jose Bonifacio            863
SAO - Batista                   861
GDL - Ciudad del Sol            850
RIO - Tijuca                    806
RIO - Botafogo                  798
RIO - Barra                     795
MEX - Lindavista                766
BHZ - Tancredo Neves            727
BOG - Castellana                699
MEX - Insurgentes Sur           692
BOG - San Felipe                670
LIM - Surquillo                 657
SCL - Recoleta                  656
MED - DModa                 

In [42]:
# getting exchange rate
#gc = pygsheets.authorize(service_file=g_sheets_creds_path)
#ex = gc.open_by_key('1UNylsHToNDVNfrTC9Le5oRu0rUlA1edkW8RuSAPPtV8')
#exws = ex.worksheet('title','latam_monthly')
#latam_monthly = exws.get_as_df(has_header=True,start="A1", end=None,empty_value='')
#latam_monthly = pd.melt(latam_monthly, id_vars='Month', var_name='country', value_name='exchange_rate')
#latam_monthly.Month = latam_monthly.Month.astype('datetime64[ns]')
#latam_monthly.exchange_rate = pd.to_numeric(latam_monthly.exchange_rate ,errors='coerce')
#latam_monthly.rename(columns={'Month': 'created_month'}, inplace=True)

# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
#ss = gc.open_by_key('1rEQZXWO2GfxKWPApn7OFQ-JTOUTe_7PdAsOd8Ma2fEI')

exchange = {'Mexico':'MXN',
           'Panama':'USD',
           'Colombia':'COP',
           'Ecuador':'USD',
           'Costa Rica':'CRC',
           'Peru':'PEN',
           'Chile':'CLP',
           'Brazil':'BRL'
           }

#latam_monthly['currency_code'] = latam_monthly.country.map(exchange)
#latam_monthly = latam_monthly[['created_month', 'country', 'currency_code', 'exchange_rate']]

#ws = ss.worksheet('title','exchange_rate')

print('Clearing Range ...')
#ws.clear(start='B', end='D')
print('Range CLEARED')

print('Pasting Info ...')
#ws.set_dataframe(latam_monthly, start="B1",copy_head=True, extend=True, nan='')
print('Exchange Rate UPDATED!!')

Clearing Range ...
Range CLEARED
Pasting Info ...
Exchange Rate UPDATED!!


In [43]:
import pygsheets
#tipo de cambio historico
# getting exchange rate
gc = pygsheets.authorize(custom_credentials=credentials_sheets)
ex = gc.open_by_key('1h_lgOO2wf89txM2jX5ZXS64DizqewyPUReI_0-zP2M0')
exws = ex.worksheet('title','latam_monthly')
latam_monthly = exws.get_as_df(has_header=True,start="A1", end=None,empty_value='')

In [44]:
latam_monthly['exchange_rate']=latam_monthly['exchange_rate'].replace('#N/A','0')

In [45]:
latam_monthly['exchange_rate']=latam_monthly['exchange_rate'].replace('','0')

In [46]:

latam_monthly['exchange_rate']=latam_monthly['exchange_rate'].astype('float64') 

In [47]:
latam_monthly['created_month']=pd.to_datetime(latam_monthly['created_month'], errors='coerce')

In [48]:
latam_monthly

,created_month,country,currency_code,exchange_rate
0,2019-01-01,Mexico,MXN,18.80
1,2019-02-01,Mexico,MXN,19.59
2,2019-03-01,Mexico,MXN,22.26
3,2019-04-01,Mexico,MXN,24.20
4,2019-05-01,Mexico,MXN,23.47
...,...,...,...,...
679,2024-12-01,USA,USD,0.00
680,2025-01-01,USA,USD,0.00
681,2025-02-01,USA,USD,0.00
682,2025-03-01,USA,USD,0.00


In [49]:
final_exp_2 = final_exp.merge(latam_monthly, on=['created_month','country'], how='left')
final_exp_2['all_gmv_usd'] = final_exp_2.all_gmv / final_exp_2.exchange_rate
final_exp_2['cancelled_gmv_usd'] = final_exp_2.cancelled_gmv / final_exp_2.exchange_rate
final_exp_2['gmv_usd'] = final_exp_2.gmv / final_exp_2.exchange_rate
final_exp_2['aov_usd'] = final_exp_2.aov / final_exp_2.exchange_rate


final_exp_2.drop(['exchange_rate'], axis=1, inplace=True)


In [50]:
final_exp_2

,created_month,region,country,city,facility_name,first_order_Date,last_order_Date,organization_name,organization_id,tenant_type,...,gmv,aov,all_gmv_usd,cancelled_gmv_usd,net_payout,discount,net_gmv,currency_code,gmv_usd,aov_usd
0,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,NaT,NaT,Basilico Pizza - KC,NaN,DELIVERY,...,369.91,33.628182,71.827184,0.000000,369.91,0.00,369.91,BRL,71.827184,6.529744
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,NaT,NaT,Philippes Pizza - KC,NaN,DELIVERY,...,4027.10,57.530000,781.961165,0.000000,4027.10,0.00,4027.10,BRL,781.961165,11.170874
2,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,NaT,NaT,Saú Culinária Saudável,NaN,DELIVERY,...,445.60,40.509091,86.524272,0.000000,445.60,0.00,445.60,BRL,86.524272,7.865843
3,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,74.80,74.800000,14.524272,0.000000,74.80,0.00,74.80,BRL,14.524272,14.524272
4,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Dino,NaT,NaT,American Burger,NaN,DELIVERY,...,22590.02,33.269543,4386.411650,0.000000,22590.02,0.00,22590.02,BRL,4386.411650,6.460106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-06-15 21:30:02,NaT,Palermo - CK,b238f8ff-e3ac-4c62-a940-f6eac0ed5a57,DELIVERY,...,7859.00,45.959064,2131.756757,7.702703,0.00,262.20,7596.80,PEN,2124.054054,12.421369
37427,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-06-24 19:10:19,NaT,MEGABITE,e27bf782-a8b4-46e8-8643-c5ea5e6cdfdd,DELIVERY,...,3026.08,48.033016,830.164865,12.305405,0.00,553.61,2472.47,PEN,817.859459,12.981896
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-08-23 19:08:37,NaT,Pikalo - CK,6a8d7b6c-7de9-4c07-a3e2-a4f6b630b2dc,DELIVERY,...,0.00,0.000000,107.162162,107.162162,0.00,0.00,0.00,PEN,0.000000,0.000000
37429,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2023-09-08 20:16:09,NaT,Corporacion Baguetera SAC,ad0cc4ed-50d2-4489-a6bb-a463dbba000d,DELIVERY,...,832.00,48.941176,224.864865,0.000000,0.00,0.00,0.00,PEN,224.864865,13.227345


In [51]:
final_exp_3 = final_exp_2.sort_values(by=['country','organization_name', 'organization_id'], ascending=True, na_position='last').reset_index(drop=True)

#final_exp_3.organization_id = final_exp_3.organization_id.ffill()



In [52]:
final_exp_3

,created_month,region,country,city,facility_name,first_order_Date,last_order_Date,organization_name,organization_id,tenant_type,...,gmv,aov,all_gmv_usd,cancelled_gmv_usd,net_payout,discount,net_gmv,currency_code,gmv_usd,aov_usd
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,NaT,NaT,A Casa da Severina,NaN,DELIVERY,...,91.90,91.900000,18.453815,0.000000,91.90,0.00,91.90,BRL,18.453815,18.453815
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,74.80,74.800000,14.524272,0.000000,74.80,0.00,74.80,BRL,14.524272,14.524272
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,1401.93,58.413750,262.042991,0.000000,1401.93,0.00,1401.93,BRL,262.042991,10.918458
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,2075.60,74.128571,382.952030,0.000000,2075.60,0.00,2075.60,BRL,382.952030,13.676858
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,32569.68,73.520722,5774.765957,0.000000,32569.68,0.00,32569.68,BRL,5774.765957,13.035589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,2022-05-28 00:03:21,2023-09-13 19:46:45,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,11883.30,54.510550,3306.837838,95.135135,0.00,1360.00,10523.30,PEN,3211.702703,14.732581
37427,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,2022-05-28 18:35:52,2023-09-13 18:56:14,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,68538.40,54.743131,18913.891892,390.000000,0.00,6120.75,62417.65,PEN,18523.891892,14.795441
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2022-07-15 00:21:40,NaT,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,70485.75,58.397473,19213.581081,163.378378,0.00,6015.45,64470.30,PEN,19050.202703,15.783101
37429,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2022-07-15 01:47:58,NaT,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,9775.10,56.178736,2651.918919,10.000000,0.00,775.00,9000.10,PEN,2641.918919,15.183442


In [53]:
np.datetime64('today')

numpy.datetime64('2023-09-13')

In [54]:
final_exp_3['created_month']=pd.to_datetime(final_exp_3['created_month'])

In [55]:
final_exp_3

,created_month,region,country,city,facility_name,first_order_Date,last_order_Date,organization_name,organization_id,tenant_type,...,gmv,aov,all_gmv_usd,cancelled_gmv_usd,net_payout,discount,net_gmv,currency_code,gmv_usd,aov_usd
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,NaT,NaT,A Casa da Severina,NaN,DELIVERY,...,91.90,91.900000,18.453815,0.000000,91.90,0.00,91.90,BRL,18.453815,18.453815
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,74.80,74.800000,14.524272,0.000000,74.80,0.00,74.80,BRL,14.524272,14.524272
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,1401.93,58.413750,262.042991,0.000000,1401.93,0.00,1401.93,BRL,262.042991,10.918458
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,2075.60,74.128571,382.952030,0.000000,2075.60,0.00,2075.60,BRL,382.952030,13.676858
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,32569.68,73.520722,5774.765957,0.000000,32569.68,0.00,32569.68,BRL,5774.765957,13.035589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,2022-05-28 00:03:21,2023-09-13 19:46:45,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,11883.30,54.510550,3306.837838,95.135135,0.00,1360.00,10523.30,PEN,3211.702703,14.732581
37427,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,2022-05-28 18:35:52,2023-09-13 18:56:14,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,68538.40,54.743131,18913.891892,390.000000,0.00,6120.75,62417.65,PEN,18523.891892,14.795441
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2022-07-15 00:21:40,NaT,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,70485.75,58.397473,19213.581081,163.378378,0.00,6015.45,64470.30,PEN,19050.202703,15.783101
37429,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2022-07-15 01:47:58,NaT,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,9775.10,56.178736,2651.918919,10.000000,0.00,775.00,9000.10,PEN,2641.918919,15.183442


In [56]:
final_exp_4 = final_exp_3[final_exp_3.created_month <= np.datetime64('today')]

In [57]:
final_exp_4

,created_month,region,country,city,facility_name,first_order_Date,last_order_Date,organization_name,organization_id,tenant_type,...,gmv,aov,all_gmv_usd,cancelled_gmv_usd,net_payout,discount,net_gmv,currency_code,gmv_usd,aov_usd
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,NaT,NaT,A Casa da Severina,NaN,DELIVERY,...,91.90,91.900000,18.453815,0.000000,91.90,0.00,91.90,BRL,18.453815,18.453815
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,74.80,74.800000,14.524272,0.000000,74.80,0.00,74.80,BRL,14.524272,14.524272
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,1401.93,58.413750,262.042991,0.000000,1401.93,0.00,1401.93,BRL,262.042991,10.918458
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,2075.60,74.128571,382.952030,0.000000,2075.60,0.00,2075.60,BRL,382.952030,13.676858
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,NaT,NaT,#N/A,NaN,DELIVERY,...,32569.68,73.520722,5774.765957,0.000000,32569.68,0.00,32569.68,BRL,5774.765957,13.035589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,2022-05-28 00:03:21,2023-09-13 19:46:45,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,11883.30,54.510550,3306.837838,95.135135,0.00,1360.00,10523.30,PEN,3211.702703,14.732581
37427,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,2022-05-28 18:35:52,2023-09-13 18:56:14,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,68538.40,54.743131,18913.891892,390.000000,0.00,6120.75,62417.65,PEN,18523.891892,14.795441
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2022-07-15 00:21:40,NaT,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,70485.75,58.397473,19213.581081,163.378378,0.00,6015.45,64470.30,PEN,19050.202703,15.783101
37429,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,2022-07-15 01:47:58,NaT,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,...,9775.10,56.178736,2651.918919,10.000000,0.00,775.00,9000.10,PEN,2641.918919,15.183442


In [58]:
final_exp_5 = final_exp_4[[c for c in latam_2.columns]]

In [59]:
final_exp_5

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,A Casa da Severina,NaN,DELIVERY,1,0,...,91.900000,18.453815,0.000000,18.453815,18.453815,91.90,0.00,91.90,NaT,NaT
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,1,0,...,74.800000,14.524272,0.000000,14.524272,14.524272,74.80,0.00,74.80,NaT,NaT
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,24,0,...,58.413750,262.042991,0.000000,262.042991,10.918458,1401.93,0.00,1401.93,NaT,NaT
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,28,0,...,74.128571,382.952030,0.000000,382.952030,13.676858,2075.60,0.00,2075.60,NaT,NaT
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,443,0,...,73.520722,5774.765957,0.000000,5774.765957,13.035589,32569.68,0.00,32569.68,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,224,6,...,54.510550,3306.837838,95.135135,3211.702703,14.732581,0.00,1360.00,10523.30,2022-05-28 00:03:21,2023-09-13 19:46:45
37427,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1280,28,...,54.743131,18913.891892,390.000000,18523.891892,14.795441,0.00,6120.75,62417.65,2022-05-28 18:35:52,2023-09-13 18:56:14
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1220,13,...,58.397473,19213.581081,163.378378,19050.202703,15.783101,0.00,6015.45,64470.30,2022-07-15 00:21:40,NaT
37429,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,175,1,...,56.178736,2651.918919,10.000000,2641.918919,15.183442,0.00,775.00,9000.10,2022-07-15 01:47:58,NaT


In [60]:
final_exp_6 = final_exp_5.sort_values(by=['created_month','country', 'city', 'facility_name'], ascending=True, na_position='last').reset_index(drop=True)

final_exp_6['net_gmv'] = final_exp_6.gmv - final.discount

In [61]:
final_exp_6

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,Basilico Pizza - KC,NaN,DELIVERY,11,0,...,33.628182,71.827184,0.000000,71.827184,6.529744,369.91,0.00,-39530.09,NaT,NaT
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,Philippes Pizza - KC,NaN,DELIVERY,70,0,...,57.530000,781.961165,0.000000,781.961165,11.170874,4027.10,0.00,-167772.90,NaT,NaT
2,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Arthur Guimaraes,Saú Culinária Saudável,NaN,DELIVERY,11,0,...,40.509091,86.524272,0.000000,86.524272,7.865843,445.60,0.00,NaN,NaT,NaT
3,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,1,0,...,74.800000,14.524272,0.000000,14.524272,14.524272,74.80,0.00,-20825.20,NaT,NaT
4,2020-12-01,LATAM,Brazil,Sao Paulo,SAO - Dino,American Burger,NaN,DELIVERY,679,0,...,33.269543,4386.411650,0.000000,4386.411650,6.460106,22590.02,0.00,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37425,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,Virtual Rockets,5c66df72-7130-4c4c-806f-8ae2b1f18c95,DELIVERY,6,6,...,0.000000,52.243243,52.243243,0.000000,0.000000,0.00,0.00,0.00,2022-08-26 17:50:42,NaT
37426,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,Virtual Rockets,5c66df72-7130-4c4c-806f-8ae2b1f18c95,DELIVERY,1,1,...,0.000000,6.729730,6.729730,0.000000,0.000000,0.00,0.00,0.00,2023-01-22 18:16:32,NaT
37427,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,Wingman - VK,d074906b-5a62-44a2-80af-5a130324f87b,DELIVERY,70,1,...,71.917391,1359.810811,18.648649,1341.162162,19.437133,0.00,98.00,4962.30,2022-07-11 21:59:23,NaT
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1220,13,...,58.397473,19213.581081,163.378378,19050.202703,15.783101,0.00,6015.45,70485.75,2022-07-15 00:21:40,NaT


In [62]:
centrito_nano = [{'created_month':datetime(2021, 10, 1), 
'region':'LATAM',
'country':'Mexico', 
'city':'Monterrey',
'facility_name':'MTY - Centrito NK', 
'organization_name':np.nan,
'organization_id':np.nan,
'tenant_type':'DELIVERY',
'all_orders':6774,
'cancelled_orders':0,
'orders':6774, 
 'all_gmv':2147358,
'cancelled_gmv':0,
'gmv':2147358,
'aov':np.nan,
'all_gmv_usd':104997, 
'cancelled_gmv_usd':0, 
'gmv_usd':104997,
'aov_usd':np.nan,
'net_payout':np.nan,
'discount':np.nan, 
'net_gmv':np.nan,
 'first_order_Date':np.nan},

{'created_month':datetime(2021, 11, 1), 

'region':'LATAM',
'country':'Mexico', 
'city':'Monterrey',
'facility_name':'MTY - Centrito NK', 
'organization_name':np.nan,
'organization_id':np.nan,
'tenant_type':'DELIVERY',
'all_orders':3387,
'cancelled_orders':0,
'orders':3387, 
 'all_gmv':1073679,
'cancelled_gmv':0,
'gmv':1073679,
'aov':np.nan,
'all_gmv_usd':51439, 
'cancelled_gmv_usd':0, 
'gmv_usd':51439,
'aov_usd':np.nan,
'net_payout':np.nan,
'discount':np.nan, 
'net_gmv':np.nan,
'first_order_Date':np.nan,
'last_order_Date':np.nan}]


In [63]:
for month in centrito_nano:
    final_exp_6 = final_exp_5.append(month, ignore_index=True)

/tmp/ipykernel_3610/1411103330.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_exp_6 = final_exp_5.append(month, ignore_index=True)
/tmp/ipykernel_3610/1411103330.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_exp_6 = final_exp_5.append(month, ignore_index=True)


In [64]:
final_exp_6

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,A Casa da Severina,NaN,DELIVERY,1,0,...,91.900000,18.453815,0.000000,18.453815,18.453815,91.90,0.00,91.90,NaT,NaT
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,1,0,...,74.800000,14.524272,0.000000,14.524272,14.524272,74.80,0.00,74.80,NaT,NaT
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,24,0,...,58.413750,262.042991,0.000000,262.042991,10.918458,1401.93,0.00,1401.93,NaT,NaT
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,28,0,...,74.128571,382.952030,0.000000,382.952030,13.676858,2075.60,0.00,2075.60,NaT,NaT
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,443,0,...,73.520722,5774.765957,0.000000,5774.765957,13.035589,32569.68,0.00,32569.68,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1280,28,...,54.743131,18913.891892,390.000000,18523.891892,14.795441,0.00,6120.75,62417.65,2022-05-28 18:35:52,2023-09-13 18:56:14
37427,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1220,13,...,58.397473,19213.581081,163.378378,19050.202703,15.783101,0.00,6015.45,64470.30,2022-07-15 00:21:40,NaT
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,175,1,...,56.178736,2651.918919,10.000000,2641.918919,15.183442,0.00,775.00,9000.10,2022-07-15 01:47:58,NaT
37429,2022-04-01,LATAM,Peru,Lima,LIM - Magdalena,NaN,4523cf2a-dadb-4df5-87fe-515d2a5d19eb,DELIVERY,2,0,...,38.500000,20.754717,0.000000,20.754717,10.377358,0.00,0.00,0.00,2022-03-17 17:33:46,2022-04-07 14:48:21


In [65]:
final_exp_6.created_month.value_counts()

2023-02-01    2012
2023-04-01    1998
2023-03-01    1987
2023-06-01    1975
2023-05-01    1970
2023-07-01    1963
2023-08-01    1935
2022-07-01    1917
2022-10-01    1810
2022-09-01    1807
2022-11-01    1791
2022-12-01    1785
2022-08-01    1781
2023-01-01    1740
2022-06-01    1719
2022-05-01    1635
2023-09-01    1602
2022-03-01    1503
2022-04-01    1485
2022-02-01    1413
2022-01-01    1395
2021-10-01      23
2021-02-01      23
2021-04-01      19
2021-03-01      19
2021-01-01      19
2021-12-01      17
2021-05-01      16
2021-11-01      15
2021-09-01      14
2020-12-01      12
2021-07-01      11
2021-08-01      10
2021-06-01      10
Name: created_month, dtype: int64

In [66]:
final_exp_6

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,A Casa da Severina,NaN,DELIVERY,1,0,...,91.900000,18.453815,0.000000,18.453815,18.453815,91.90,0.00,91.90,NaT,NaT
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,1,0,...,74.800000,14.524272,0.000000,14.524272,14.524272,74.80,0.00,74.80,NaT,NaT
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,24,0,...,58.413750,262.042991,0.000000,262.042991,10.918458,1401.93,0.00,1401.93,NaT,NaT
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,28,0,...,74.128571,382.952030,0.000000,382.952030,13.676858,2075.60,0.00,2075.60,NaT,NaT
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,443,0,...,73.520722,5774.765957,0.000000,5774.765957,13.035589,32569.68,0.00,32569.68,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1280,28,...,54.743131,18913.891892,390.000000,18523.891892,14.795441,0.00,6120.75,62417.65,2022-05-28 18:35:52,2023-09-13 18:56:14
37427,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1220,13,...,58.397473,19213.581081,163.378378,19050.202703,15.783101,0.00,6015.45,64470.30,2022-07-15 00:21:40,NaT
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,175,1,...,56.178736,2651.918919,10.000000,2641.918919,15.183442,0.00,775.00,9000.10,2022-07-15 01:47:58,NaT
37429,2022-04-01,LATAM,Peru,Lima,LIM - Magdalena,NaN,4523cf2a-dadb-4df5-87fe-515d2a5d19eb,DELIVERY,2,0,...,38.500000,20.754717,0.000000,20.754717,10.377358,0.00,0.00,0.00,2022-03-17 17:33:46,2022-04-07 14:48:21


In [67]:
final_exp_6

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,A Casa da Severina,NaN,DELIVERY,1,0,...,91.900000,18.453815,0.000000,18.453815,18.453815,91.90,0.00,91.90,NaT,NaT
1,2020-12-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,1,0,...,74.800000,14.524272,0.000000,14.524272,14.524272,74.80,0.00,74.80,NaT,NaT
2,2021-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,24,0,...,58.413750,262.042991,0.000000,262.042991,10.918458,1401.93,0.00,1401.93,NaT,NaT
3,2021-02-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,28,0,...,74.128571,382.952030,0.000000,382.952030,13.676858,2075.60,0.00,2075.60,NaT,NaT
4,2021-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,443,0,...,73.520722,5774.765957,0.000000,5774.765957,13.035589,32569.68,0.00,32569.68,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37426,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1280,28,...,54.743131,18913.891892,390.000000,18523.891892,14.795441,0.00,6120.75,62417.65,2022-05-28 18:35:52,2023-09-13 18:56:14
37427,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1220,13,...,58.397473,19213.581081,163.378378,19050.202703,15.783101,0.00,6015.45,64470.30,2022-07-15 00:21:40,NaT
37428,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,175,1,...,56.178736,2651.918919,10.000000,2641.918919,15.183442,0.00,775.00,9000.10,2022-07-15 01:47:58,NaT
37429,2022-04-01,LATAM,Peru,Lima,LIM - Magdalena,NaN,4523cf2a-dadb-4df5-87fe-515d2a5d19eb,DELIVERY,2,0,...,38.500000,20.754717,0.000000,20.754717,10.377358,0.00,0.00,0.00,2022-03-17 17:33:46,2022-04-07 14:48:21


In [68]:
final_exp_6.dtypes

created_month        datetime64[ns]
region                       object
country                      object
city                         object
facility_name                object
organization_name            object
organization_id              object
tenant_type                  object
all_orders                    int64
cancelled_orders              int64
orders                        int64
all_gmv                     float64
cancelled_gmv               float64
gmv                         float64
aov                         float64
all_gmv_usd                 float64
cancelled_gmv_usd           float64
gmv_usd                     float64
aov_usd                     float64
net_payout                  float64
discount                    float64
net_gmv                     float64
first_order_Date     datetime64[ns]
last_order_Date      datetime64[ns]
dtype: object

In [69]:
np.datetime64('2022-01-01')

numpy.datetime64('2022-01-01')

In [70]:
final_exp_6['facility_name'].value_counts()

MEX - Polanco                  1800
MEX - Pedregal                 1674
MEX - Roma                     1599
SAO - Guararapes (Brooklin)    1542
LIM - Magdalena                1426
SAO - Rua do Acre              1414
SAO - Dino                     1387
MEX - Napoles                  1370
BOG - Salitre 2 EM             1141
MEX - Santa Fe                 1133
BHZ - Paracatu                 1079
SAO - Lapa Clelia               982
BOG - Atabanza                  970
GDL - Av. Mexico                951
RIO - Jose Bonifacio            863
SAO - Batista                   861
GDL - Ciudad del Sol            850
RIO - Tijuca                    806
RIO - Botafogo                  798
RIO - Barra                     795
MEX - Lindavista                766
BHZ - Tancredo Neves            727
BOG - Castellana                699
MEX - Insurgentes Sur           692
BOG - San Felipe                670
LIM - Surquillo                 657
SCL - Recoleta                  656
MED - DModa                 

In [71]:
archivo_order_volumne=final_exp_6[final_exp_6['created_month']>=np.datetime64('2022-01-01')].reset_index(drop=True)

In [72]:
archivo_order_volumne

,created_month,region,country,city,facility_name,organization_name,organization_id,tenant_type,all_orders,cancelled_orders,...,aov,all_gmv_usd,cancelled_gmv_usd,gmv_usd,aov_usd,net_payout,discount,net_gmv,first_order_Date,last_order_Date
0,2022-03-01,LATAM,Brazil,Rio de Janeiro,RIO - Jose Bonifacio,A Casa da Severina,NaN,DELIVERY,1,0,...,91.900000,18.453815,0.000000,18.453815,18.453815,91.90,0.00,91.90,NaT,NaT
1,2022-01-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,1,0,...,159.600000,28.860759,0.000000,28.860759,28.860759,159.60,0.00,159.60,NaT,NaT
2,2022-03-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,67,0,...,28.511194,383.584337,0.000000,383.584337,5.725139,1910.25,0.00,1910.25,NaT,NaT
3,2022-04-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,203,0,...,37.666749,1613.154008,0.000000,1613.154008,7.946571,7646.35,0.00,7646.35,NaT,NaT
4,2022-05-01,LATAM,Brazil,Belo Horizonte,BHZ - Paracatu,#N/A,NaN,DELIVERY,125,0,...,30.548640,771.430303,0.000000,771.430303,6.171442,3818.58,0.00,3818.58,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37218,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,224,6,...,54.510550,3306.837838,95.135135,3211.702703,14.732581,0.00,1360.00,10523.30,2022-05-28 00:03:21,2023-09-13 19:46:45
37219,2023-09-01,LATAM,Peru,Lima,LIM - Magdalena,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1280,28,...,54.743131,18913.891892,390.000000,18523.891892,14.795441,0.00,6120.75,62417.65,2022-05-28 18:35:52,2023-09-13 18:56:14
37220,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,1220,13,...,58.397473,19213.581081,163.378378,19050.202703,15.783101,0.00,6015.45,64470.30,2022-07-15 00:21:40,NaT
37221,2023-09-01,LATAM,Peru,NaN,LIM - La Molina,YOPO SAC - VK,85c4c76b-b623-451d-b5b8-e9b0f1d49cbf,DELIVERY,175,1,...,56.178736,2651.918919,10.000000,2641.918919,15.183442,0.00,775.00,9000.10,2022-07-15 01:47:58,NaT


In [73]:
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')
# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1PsOJjzhtK2zjgg349MWf9W4kI2gVmQ77LUTtfPQ7Ksw')

# select sheet

ws = ss.worksheet('title','superset_month')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(archivo_order_volumne, start="A1",copy_head=True, extend=True)
print('LATAM ORDER VOLUME UPDATED!!')


Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
LATAM ORDER VOLUME UPDATED!!


In [74]:
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')
# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1ob8eSSGTjVT_QeQB8q_i8A3yWuiw8Q5HHnT9C0B7T3Q')

# select sheet

ws = ss.worksheet('title','superset_month')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final_exp_6, start="A1",copy_head=True, extend=True)
print('LATAM ORDER VOLUME UPDATED!!')


Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
LATAM ORDER VOLUME UPDATED!!


In [75]:
#otro archivo 
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')
# 1kQyxa3QDH-MyCd1HNxtCUuCUVb36dFX_syaUBViuJoA
ss = gc.open_by_key('1XfuB4pnAt5LiZmH7zCjNochwLHFTlCqHXS2SNiKEa0U')

# select sheet

ws = ss.worksheet('title','superset')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(final_exp_6, start="A1",copy_head=True, extend=True)
print('LATAM ORDER VOLUME UPDATED file kpis!!')
#

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
LATAM ORDER VOLUME UPDATED file kpis!!


# big query

In [78]:
from ds.utilities.secrets import secrets
from google.oauth2 import service_account
from google.cloud import bigquery
s = secrets.Secrets()
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"))
client = bigquery.Client(credentials=credentials)

In [79]:
#subo a google cloud

final_exp_6.to_gbq(credentials = credentials ,
              destination_table='latam_cr_growth.Finanzas_order_volumne',
              progress_bar = True,
              if_exists='replace'
              #if_exists='append'
                        )

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_pandas_gbq/site-packages/pandas_gbq/schema.py:107: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, dtype in dataframe.dtypes.iteritems():
1it [00:05,  5.78s/it]


In [80]:
#sales force
import json
import pandas as pd
import numpy as np
import json

# UNCOMMENT IF YOU DO NOT HAVE SIMPLE_SALESFORCE INSTALLED
#!pip install simple_salesforce 
from simple_salesforce import Salesforce, SalesforceLogin, SFType


path = "/home/ruben.lopez/creds_sales_force.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo['username'],
                                                 password=loginInfo['password'],
                                                 security_token=loginInfo['security_token'],
                                                 domain='login')

sf = Salesforce(instance=instance, session_id=session_id)


In [81]:
query_sof="""
select Opportunity_ID_18__c, AccountId, SkuSku_Organization_ID__c, Opp_Owner_Role__c, Opp_Owner_Region__c, Opp_Owner_Country__c, Account_Name_Jira__c, Opportunity_Owner_Name__c, Opportunity_Company_Name__c, CreatedDate, Lead_Source_Detail__c, LeadSource, Kitchen_Country__c, Facility_ID__c, Closed_Won_Date__c, Churn_Date__c, Is_Churned__c,Onboarding_Fee__c, Deposit_Fee__c, Monthly_License_Fee__c, Monthly_Shelf_Fee_Cold_Storage__c, Monthly_Shelf_Fee_Dry_Storage__c, Monthly_Shelf_Fee_Frozen_Storage__c, Storage_Amount_Cold_CU_FT__c, Storage_Amount_Dry_CU_FT__c, Storage_Amount_Frozen_CU_FT__c, CurrencyIsoCode, RecordType.Name, Access_Date__c, Activation_Date__c, Last_Activity_Date__c, Start_Date__c, End_Date__c, Primary_Churn_Reason__c, Order_Form_URL__c, SDR_Name__c, SDR__c, Facility__c, Renewal_Term_Length__c, Amount, Processing_Fee__c, Cuisine_Type__c from Opportunity where Opp_Owner_Region__c = 'LATAM' and RecordType.Name = 'CloudRetail' and StageName = 'Closed Won'


"""

print('Getting Data...')

# Query users to dataframe
opp = pd.DataFrame(sf.query_all(query_sof)['records'])
print('Users Pulled')

opp

Getting Data...
Users Pulled


,attributes,Opportunity_ID_18__c,AccountId,SkuSku_Organization_ID__c,Opp_Owner_Role__c,Opp_Owner_Region__c,Opp_Owner_Country__c,Account_Name_Jira__c,Opportunity_Owner_Name__c,Opportunity_Company_Name__c,...,End_Date__c,Primary_Churn_Reason__c,Order_Form_URL__c,SDR_Name__c,SDR__c,Facility__c,Renewal_Term_Length__c,Amount,Processing_Fee__c,Cuisine_Type__c
0,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D8f9RQAR,0016T000035CBhjQAG,None,Account Executive CR,LATAM,Brazil,Boleria Love Sugar,Gabriel Henrique De Godoy,BOLERIA LOVE SUGAR LTDA,...,2024-10-10,None,https://drive.google.com/drive/folders/11PsxKg...,None,None,0015G00001YnStAQAV,NaN,5984.0,10.0,None
1,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D8e46QAB,0016T000030PLAiQAO,None,CR - Account Executive (AE),LATAM,Brazil,Palanda,Arthur Coelho Andrade,LUIS CARLOS PEREZ COVO,...,2023-03-18,None,https://drive.google.com/drive/u/0/folders/1pA...,None,None,0015G00001Vbgz1QAB,NaN,NaN,7.0,None
2,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D8Y6yQAF,0016T000035FUZeQAO,None,Account Executive CR,LATAM,Brazil,KiiManá,Pamela Casimiro,51.029.233 LILIAN DINIZ APOLINARIO,...,2024-09-28,None,https://drive.google.com/drive/u/1/folders/15p...,None,None,001f400001MF2ckAAD,NaN,16160.0,10.0,None
3,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D8Y7IQAV,0016T000035FUZeQAO,None,Account Executive CR,LATAM,Brazil,KiiManá,Pamela Casimiro,51.029.233 LILIAN DINIZ APOLINARIO,...,2024-10-17,None,https://drive.google.com/drive/u/1/folders/15p...,None,None,001f400001HaBcsAAF,NaN,16160.0,10.0,None
4,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D8Y7JQAV,0016T000035FUZeQAO,None,Account Executive CR,LATAM,Brazil,KiiManá,Pamela Casimiro,51.029.233 LILIAN DINIZ APOLINARIO,...,2024-10-17,None,https://drive.google.com/drive/u/1/folders/15p...,None,None,001f400001NYEY5AAP,NaN,16160.0,10.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D7KQAQA3,0016T00002wT7GpQAK,None,Account Executive CR,LATAM,Brazil,Smart Pay Store,Pamela Casimiro,SAMUEL MONTEIRO GOMES 12229263773,...,2024-09-11,None,https://drive.google.com/drive/u/1/folders/1kn...,Ednai Soares,None,001f400001NXiw4AAD,NaN,260.0,0.0,None
2771,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D7HvmQAF,0016T000035Dv7jQAC,None,CR - Account Executive (AE),LATAM,Mexico,Cinnamon Deli,Gisell Ariatna Dominguez Rangel,Filomena Pérez Cruz,...,2024-08-28,None,https://app.pandadoc.com/a/#/documents/zxV66zC...,Lizeth Lamas,None,001f400001Kf5ndAAB,12.0,20600.0,8.0,None
2772,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D7JfYQAV,0015G00001aCr0bQAC,None,Account Executive CR,LATAM,Brazil,Matheus Patisserie Sant'Anna,Gabriel Henrique De Godoy,MATHEUS SANT'ANNA PATISSERIE EIRELI,...,2024-09-10,None,https://drive.google.com/drive/u/0/folders/1TM...,None,None,0015G00001YnSs7QAF,NaN,NaN,5.0,None
2773,"{'type': 'Opportunity', 'url': '/services/data...",0066T00001D7JpOQAV,0016T00002xKxPQQA0,None,CR - CSM,LATAM,Mexico,CBD Life,Daniela Pulido,CBD Life S.A de C.V,...,2023-07-24,None,https://drive.google.com/drive/folders/1cR2KE0...,Diandra Gutiérrez,None,001f400001HXpHoAAL,NaN,9800.0,0.0,None


In [82]:
opp['MRR_local']=opp['Monthly_Shelf_Fee_Cold_Storage__c']+opp['Monthly_Shelf_Fee_Dry_Storage__c']+opp['Monthly_Shelf_Fee_Frozen_Storage__c']

In [83]:
opp['MRR_local']=opp['MRR_local'].fillna(0)

In [84]:
opp['Closed_Won_Date__c']=pd.to_datetime(opp['Closed_Won_Date__c'], errors='coerce')

In [85]:
opp['close_won_month']=opp['Closed_Won_Date__c'].dt.to_period('M').dt.to_timestamp()

In [86]:
latam_monthly

,created_month,country,currency_code,exchange_rate
0,2019-01-01,Mexico,MXN,18.80
1,2019-02-01,Mexico,MXN,19.59
2,2019-03-01,Mexico,MXN,22.26
3,2019-04-01,Mexico,MXN,24.20
4,2019-05-01,Mexico,MXN,23.47
...,...,...,...,...
679,2024-12-01,USA,USD,0.00
680,2025-01-01,USA,USD,0.00
681,2025-02-01,USA,USD,0.00
682,2025-03-01,USA,USD,0.00


In [87]:
opp.columns

Index(['attributes', 'Opportunity_ID_18__c', 'AccountId',
       'SkuSku_Organization_ID__c', 'Opp_Owner_Role__c', 'Opp_Owner_Region__c',
       'Opp_Owner_Country__c', 'Account_Name_Jira__c',
       'Opportunity_Owner_Name__c', 'Opportunity_Company_Name__c',
       'CreatedDate', 'Lead_Source_Detail__c', 'LeadSource',
       'Kitchen_Country__c', 'Facility_ID__c', 'Closed_Won_Date__c',
       'Churn_Date__c', 'Is_Churned__c', 'Onboarding_Fee__c', 'Deposit_Fee__c',
       'Monthly_License_Fee__c', 'Monthly_Shelf_Fee_Cold_Storage__c',
       'Monthly_Shelf_Fee_Dry_Storage__c',
       'Monthly_Shelf_Fee_Frozen_Storage__c', 'Storage_Amount_Cold_CU_FT__c',
       'Storage_Amount_Dry_CU_FT__c', 'Storage_Amount_Frozen_CU_FT__c',
       'CurrencyIsoCode', 'RecordType', 'Access_Date__c', 'Activation_Date__c',
       'Last_Activity_Date__c', 'Start_Date__c', 'End_Date__c',
       'Primary_Churn_Reason__c', 'Order_Form_URL__c', 'SDR_Name__c', 'SDR__c',
       'Facility__c', 'Renewal_Term_Length_

In [88]:
#opps_2=opp.merge(latam_monthly, left_on=['close_won_month','Kitchen_Country__c'], 
         # right_on=['created_month','country'],how='left')

In [89]:
#opps_2['MRR_usd_Exchange_rate']=(opps_2['MRR_local']/opps_2['exchange_rate']).replace(np.inf,0).fillna(0)

In [90]:
#opps_2.columns

In [91]:
#opps_exp=opps_2[['Opportunity_ID_18__c', 'AccountId',
       #'SkuSku_Organization_ID__c', 'Opp_Owner_Role__c', 'Opp_Owner_Region__c',
       #'Opp_Owner_Country__c', 'Account_Name_Jira__c',
       #'Opportunity_Owner_Name__c', 'Opportunity_Company_Name__c',
       #'CreatedDate', 'Lead_Source_Detail__c', 'LeadSource',
       #'Kitchen_Country__c', 'Facility_ID__c', 'Closed_Won_Date__c',
       #'Churn_Date__c', 'Is_Churned__c', 'Onboarding_Fee__c', 'Deposit_Fee__c',
       #'Monthly_License_Fee__c', 'Monthly_Shelf_Fee_Cold_Storage__c',
       #'Monthly_Shelf_Fee_Dry_Storage__c',
       #'Monthly_Shelf_Fee_Frozen_Storage__c', 'Storage_Amount_Cold_CU_FT__c',
       #'Storage_Amount_Dry_CU_FT__c', 'Storage_Amount_Frozen_CU_FT__c',
       #'CurrencyIsoCode', 'RecordType', 'Access_Date__c', 'Activation_Date__c',
      # 'Last_Activity_Date__c', 'Start_Date__c', 'End_Date__c',
       #'Primary_Churn_Reason__c', 'Order_Form_URL__c', 'SDR_Name__c', 'SDR__c',
      # 'Facility__c', 'Renewal_Term_Length__c', 'Amount', 'Processing_Fee__c',
      # 'Cuisine_Type__c', 'MRR_local', 'close_won_month',
                # 'exchange_rate', 'MRR_usd_Exchange_rate']]

In [108]:

#opps_exp.to_gbq(credentials = credentials ,
              #destination_table='latam_cr_growth.ops_sof',
             # progress_bar = True,
              #if_exists='replace'
              #if_exists='append'
                      #  )